<a href="https://www.kaggle.com/code/averma111/pytorch-lstm?scriptVersionId=133382703" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [27]:
%%capture
!pip install torchmetrics

In [28]:
%%capture
!pip install torchviz

In [29]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from collections import defaultdict
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
warnings.filterwarnings('ignore')
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [30]:
import torch
import torch.nn.functional as F
from torchviz import make_dot
from torchmetrics.classification import BinaryF1Score

In [31]:
class Acquisition:
    
    def __init__(self,root_path):
        self.root_path = root_path
        
    def get_dataframe(self,filename):
        return pd.read_csv(os.path.join(self.root_path,filename),encoding='UTF-8')
    
    def summary(self,text, df):
        summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
        summary['null'] = df.isnull().sum()
        summary['unique'] = df.nunique()
        summary['min'] = df.min()
        summary['median'] = df.median()
        summary['max'] = df.max()
        summary['mean'] = df.mean()
        summary['std'] = df.std()
        summary['duplicate'] = df.duplicated().sum()
        return summary
    
    def get_dataset_info(self,df):
            return df.info()

In [32]:
acq = Acquisition(root_path='/kaggle/input/nlp-getting-started')
train = acq.get_dataframe(filename='train.csv')
train=train.set_index('id')
train.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [33]:
acq.summary('text',train)

,dtypes,null,unique,min,median,max,mean,std,duplicate
keyword,object,61,221,NaN,NaN,NaN,NaN,NaN,52
location,object,2533,3341,NaN,NaN,NaN,NaN,NaN,52
text,object,0,7503,! Residents Return To Destroyed Homes As Washi...,NaN,åÈMGN-AFRICAå¨ pin:263789F4 åÈ Correction: Ten...,NaN,NaN,52
target,int64,0,2,0,0.0,1,0.42966,0.49506,52


In [45]:
class Token:
    
    def __init__(self):
        self.positive_dict = {}
        self.negative_dict = {}
    
    def tokenize(self,column):
        tokens = nltk.word_tokenize(column)
        return [w for w in tokens if w.isalpha()] 
    
    def create_token(self,df):
        df['tokenized_text'] = df.apply(lambda x: self.tokenize(x['text']) , axis=1)
        return df
    
    def postive_token(self,df):
        return df[df['target']==1]
    
    def negative_token(self,df):
        return df[df['target']==0]
    
    def top_10_postive_words(self,df):
        
        
    
    
token = Token()
train_tokenized=token.create_token(train)
train_positive = token.postive_token(train_tokenized)
train_negative = token.negative_token(train_tokenized)


,keyword,location,text,target,tokenized_text
id,,,,,
23,NaN,NaN,What's up man?,0,"[What, up, man]"
24,NaN,NaN,I love fruits,0,"[I, love, fruits]"
25,NaN,NaN,Summer is lovely,0,"[Summer, is, lovely]"
26,NaN,NaN,My car is so fast,0,"[My, car, is, so, fast]"
28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0,"[What, a, goooooooaaaaaal]"
...,...,...,...,...,...
10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,"[engineshed, Great, atmosphere, at, the, Briti..."
10834,wrecked,NaN,Cramer: Iger's 3 words that wrecked Disney's s...,0,"[Cramer, Iger, words, that, wrecked, Disney, s..."
10837,NaN,NaN,These boxes are ready to explode! Exploding Ki...,0,"[These, boxes, are, ready, to, explode, Explod..."


In [35]:
corpus = create_corpus(1)

dic = defaultdict(int)

for word in corpus:
    if word in stop:
        dic[word] += 1
        
top = sorted(dic.items(), key=lambda x: x[1], reverse=True)[:10]

x, y = zip(*top)

plt.title('With disaster')
plt.bar(x, y, color='green')

NameError: name 'create_corpus' is not defined